张量学习

In [1]:
import torch

torch.empty(5, 3)



tensor([[-5.1021e-28,  4.5902e-41, -5.1021e-28],
        [ 4.5902e-41,  3.1777e-37,  0.0000e+00],
        [ 3.1777e-37,  0.0000e+00, -1.7155e-31],
        [ 4.5902e-41, -1.4822e-31,  4.5902e-41],
        [-1.7149e-31,  4.5902e-41, -1.4832e-31]])

查看是否在GPU上


In [3]:
import torch

torch.cuda.is_available()

torch.cuda.get_device_name(0)


'NVIDIA GeForce RTX 4060 Laptop GPU'

张量计算

In [4]:
import torch

x = torch.tensor([5.5, 3,1], device='cuda', dtype=torch.bfloat16)
y = torch.tensor([1,2,3], device='cuda', dtype=torch.bfloat16)

print(x + y)

print(torch.add(x, y))



tensor([6.5000, 5.0000, 4.0000], device='cuda:0', dtype=torch.bfloat16)
tensor([6.5000, 5.0000, 4.0000], device='cuda:0', dtype=torch.bfloat16)


tensor运算gpu速度远超cpu

In [6]:
import torch
import timeit

x = torch.rand(1000, 1000).to('cpu')
y = torch.rand(1000, 1000).to('cuda')

print(timeit.timeit(lambda: x.mm(x), number=5000))

print(timeit.timeit(lambda: y.mm(y), number=5000))



26.561752285000694
1.746653828999115


自动微分，求导，梯度计算

In [8]:
import torch

x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
z = (x + y) * (x - y)

print

z.backward()

print(x.grad, y.grad)   



tensor(2.) tensor(-4.)


定义一个神经网络, 该神经网络完成图像的分类

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

train_data = datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=ToTensor()
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.Dropout(0.2)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader, start=1):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


model = NeuralNetwork().to(device)

learning_rate = 1e-3
batch_size = 64
epochs = 5

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")



Using device: cuda


100%|██████████| 26421880/26421880 [00:16<00:00, 1593341.74it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 188050.10it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:10<00:00, 433387.26it/s] 


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 28561212.95it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch 1
-------------------------------
loss: 0.751741  [ 6400/60000]
loss: 0.912375  [12800/60000]
loss: 0.711512  [19200/60000]
loss: 0.617990  [25600/60000]
loss: 0.752439  [32000/60000]
loss: 0.677582  [38400/60000]
loss: 0.733120  [44800/60000]
loss: 0.590623  [51200/60000]
loss: 0.827755  [57600/60000]
Test Error: 
 Accuracy: 75.3%, Avg loss: 0.009887 

Epoch 2
-------------------------------
loss: 0.527911  [ 6400/60000]
loss: 0.631827  [12800/60000]
loss: 0.661113  [19200/60000]
loss: 0.465484  [25600/60000]
loss: 0.688403  [32000/60000]
loss: 0.542370  [38400/60000]
loss: 0.564989  [44800/60000]
loss: 0.688316  [51200/60000]
loss: 0.464253  [57600/60000]
Test Error: 
 Accuracy: 76.2%, Avg loss: 0.009373 

Epoch 3
-------------------------------
loss: 0.467117  [ 6400/60000]
loss: 0.425402  [12800/60000]
loss: 0.470037  [19200/60000]
loss: 0.474885  [25600/60000]
loss: 0.647246  [32000/6000